# Desafio Escale

Este <i>notebook</i> traz a resolução do desafio proposto pela Escale (https://escaletech.github.io/dataplatform/data-engineer-test).

Foram criadas diversas funções, para permitir a construção de uma <i>storytelling</i> clara e objetiva.

## Setup

Instalação e importação de bibliotecas.

In [1]:
try:
    !pip install pyspark=="2.4.5" --quiet
    !pip install pandas=="1.0.4" --quiet
except:
    print("Running throw py file.")

In [2]:
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark import SparkFiles
from pyspark.sql.types import StringType, FloatType
import pyspark
import json
import pandas as pd
import numpy as np

Criação de uma sessão Spark

In [3]:
spark = SparkSession\
        .builder\
        .appName("Desafio Data Engineer Escale - Fabio Kfouri")\
        .getOrCreate()
spark

Para otimizar a resolução, foram realizados downloads dos datasets para máquina local.

Esta lógica é para identificar se esta solução esta rodando na máquina do autor, caso positivo, utilizará o dataset local, do contrário, utilizará o dataset da núvem.

Definido também a quantidade de arquivos do DataSet. A intenção do Autor é rodar a aplicação na AWS, porém, para não gerar custo desnecessario, será processado somente um arquivo do DataSet, do contrario, processará todos.

In [4]:
import os

dataPath = 'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/'
outPath = 's3://data-sprints-fk/output/'
quantity_of_datasets = 1

if 'E:\\' in os.getcwd() and 'dataEngineerTest_Escale' in os.getcwd():
    dataPath = os.getcwd() + "/data/"
    outPath = os.getcwd() + "/output/"
    quantity_of_datasets = 10

print(dataPath)
print(outPath)
print('Quantidade de Arquivos do Dataset', quantity_of_datasets)

E:\Projetos\Jobs\dataEngineerTest_Escale/data/
E:\Projetos\Jobs\dataEngineerTest_Escale/output/
Quantidade de Arquivos do Dataset 10


# Funções
Funcão de leitura do Json que retorna um Datafame Spark

In [5]:
def read_json(filename):
    return spark.read.json(dataPath + filename)

Função que converte o 30 minutos em miliegundos para ser usado como referência de delta para data em formado de <b>Epoch Time</b>.

In [6]:
def get_epoch_time_limit():
    return (30*60)*1000

Função que cria o dataFrame <b>ClickStream</b>, que atende os criterios de sessionamento defindo pelo desafio.

Para calculo do tempo de sessão foi utilizado a função <b>LAG()</b> que tem por objetivo trazer no registro corrente o dado de timestamp do registro anterior.

O autor assumiu como premissa não declarada para definir uma sessão, que além do tempo limite de 30 minutos desde a última utilização, que uma sessão precisaria considerar o <b>device_family</b> e <b>os_family</b>.

Ou seja, mesmo que não tenha excedido o tempo limite de 30 minutos, mas se for caracterizado que houve uma mudança de device_family ou os_family, trata-se de uma sessão nova.

Para a criação das Sessions_ID foi usado as funções ROW_NUMBER() para criacão ordenada de ids, e em seguida utilizada a função LAST_VALUE() para o preenchimento do session_id nos eventos da mesma sessão.

In [7]:
def get_clickstream_dataframe(df):
    overCategory = Window.partitionBy('anonymous_id','device_family','os_family','browser_family')\
                         .orderBy('anonymous_id','device_family','os_family','browser_family','device_sent_timestamp')
    
    time_limit = get_epoch_time_limit()
    
    df = df.withColumn("lag", F.lag('device_sent_timestamp', 1).over(overCategory))\
            .withColumn('delta_seg', (F.col('device_sent_timestamp') - F.col('lag'))/1000)\
            .withColumn('same_section', (F.col('device_sent_timestamp') - F.col('lag')) < time_limit) \
            .withColumn('event_time', (F.col('device_sent_timestamp')/1000).cast('timestamp'))
    
    df.createOrReplaceTempView("raw_table")
    
    df_sessioned = spark.sql("""
        with temp as (--
              SELECT anonymous_id, browser_family, device_family, device_sent_timestamp, event, event_time, n, os_family, --
                     platform, nvl(same_section,false) same_section, version,
                     case when nvl(same_section,false) = false then
                         0
                     else
                        delta_seg
                     end delta_seg,             
                     case when nvl(same_section,false) = false then
                     'session_' || ROW_NUMBER() OVER (PARTITION BY anonymous_id,nvl(same_section,false) ORDER BY device_sent_timestamp )
                     else
                      null
                     end
                     as partial_session
                FROM raw_table t --
        ), table_section_id as (--
            select anonymous_id, browser_family, delta_seg, device_family, device_sent_timestamp, event, event_time, n, os_family, --
                     platform, nvl(same_section,false) same_section, version,
                     LAST_VALUE(partial_session,True) OVER (PARTITION BY anonymous_id ORDER BY device_sent_timestamp ) || '_' || anonymous_id session_id
            from temp t
        )
        select * from table_section_id
        order by anonymous_id, device_sent_timestamp 
        """)
        
    df_sessioned.createOrReplaceTempView("clickstream")
    
    #Remove the view raw_table
    spark.catalog.dropTempView('raw_table')


Funcao que convert as saídas da Questão 2 no formato desejado pelo Desafio.

Esta função cria um dicionário que de forma incremental, vai adicionando as quantidades conforme forem sendo carregados novos datasets.

In [8]:
def convert_to_Json_challenge2(dados_pd, question):   
    for index, row in dados_pd.iterrows():
        items = str(row["collection"]).split(',')

        if not row["what"] in question:
            question[row["what"]] = {}

        for item in items:
            element = str(item).split(':')

            if not element[0] in question[row["what"]]:
                question[row["what"]][element[0]] = 0

            if len(element) > 1:
                question[row["what"]][element[0]] = float(element[1]) + question[row["what"]][element[0]]
            else:
                question[row["what"]][element[0]] = float(0) + question[row["what"]][element[0]]

                
    return question

Funcao que convert as saídas da Questão 3 no formato desejado pelo Desafio.

Esta função cria um dicionário que de forma incremental, vai calculando a mediana das medianas conforme forem sendo carregados novos datasets.

In [9]:
def convert_to_Json_challenge3(dados_pd, question):   
    for index, row in dados_pd.iterrows():

        items = str(row["collection"]).split(',')

        if not row["what"] in question:
            question[row["what"]] = {}

        for item in items:
            element = str(item).split(':')

            if not element[0] in question[row["what"]]:
                question[row["what"]][element[0]] = 0


            if len(element) > 1:
                question[row["what"]][element[0]] = np.median([ float(element[1]), question[row["what"]][element[0]] ])
            else:
                question[row["what"]][element[0]] = question[row["what"]][element[0]]

            
    return question


# Desafio 1

Calcular a quantidade total de sessões únicas por arquivo do conjunto de dados e apresentar no formato JSON.

Nesta função foi realizado um filtro para desconsiderar as sessoes abertas para descobrir o total de sessoes únicas por dataset.

In [10]:
def first_challenge(filename, question1):
    df_question_1 = spark.sql("""
        SELECT COUNT(session_id) qtd_session
        FROM clickstream
        where same_section = false
        """)
    question1[filename] = df_question_1.collect()[0]['qtd_session']
    return question1

# Desafio 2
Calcular a quantidade de sessões únicas que ocorreram em cada Browser, Sistema Operacional e Dispositivo dentro de todo o conjunto de dados.

Nesta função foi identificado as famílias (<b>browser_family</b>, <b>os_family</b> e <b>device_family</b>) e em seguida por quantidade de tipo de cada família. 

Estáo sendo consideradas somente as sessões únicas.

In [11]:
def second_challenge(question2):
    df_question_2 = spark.sql("""
        with table_temp as (--
            SELECT *
            FROM clickstream
            where same_section = false
        ), 
        table_union as (--
            SELECT 'device_family' what, device_family ref, COUNT(session_id)  qtd_session
              FROM table_temp
             group by device_family
            union
            SELECT 'os_family' what, os_family ref, COUNT(session_id)  qtd_session
              FROM table_temp
            group by os_family
            union
            SELECT 'browser_family' what, browser_family ref, COUNT(session_id)  qtd_session
              FROM table_temp
             group by browser_family --
        ),
        table_collection as (--
            select what, nvl(ref, 'Not Identified') || ':' || qtd_session collection 
            from table_union
            order by what, ref--
        )
        select what, array_join(collect_list(trim(collection)),',')  collection
        from table_collection
        group by what
        """)
    
    dados2 = df_question_2.toPandas()
    return convert_to_Json_challenge2(dados2, question2)

# Desafio 3

Calcular a mediana da duração (em segundos) entre todas sessões únicas para cada segmento.

De forma análoga ao desafio 2, esta função foram identificado as quantidades por família (<b>browser_family</b>, <b>os_family</b> e <b>device_family</b>).

Estáo sendo consideradas somente as sessões que ficaram abertas.


### Observação sobre o cálculo da Mediana
Bem, a solicitação da Mediana requer na prática que todas as durações de um segmento fossem ordenadas, para assim poder realizar a escolha do ponto central. O que na prática parece ser inviável por se tratar de um Streaming, ou seja, o dataset em um ambiente de produção crescerá ao longo do tempo.

O Autor resolveu este problema da seguinte forman:
- Calcular a mediana de cada segmento por arquivo do dataSet. 
- Cada vez que um novo arquivo era processado, uma nova mediana seria calculada.
- E usando o numpy, calculava-se a mediana das medianas, ou seja, o resultado do dataSet anterior com o resultado do Dataset atual.

Bem, a mediana neste caso não é precisa, porém para o Author, o resultado calculado atende em ordem de grandeza/aproximação a proposta do desafio. 

In [12]:
def third_challenge(question3):
    df_question_3 = spark.sql("""
        with table_temp as (--
            SELECT *
            FROM clickstream
            where same_section = true
        ), 
        table_union as (--
            SELECT 'device_family' what, device_family ref, percentile_approx(delta_seg , 0.5) median   
              FROM table_temp
             group by device_family
            union
            SELECT 'os_family' what, os_family ref, percentile_approx(delta_seg , 0.5) median 
              FROM table_temp
            group by os_family
            union
            SELECT 'browser_family' what, browser_family ref, percentile_approx(delta_seg , 0.5) median 
              FROM table_temp
             group by browser_family --
        ),
        table_collection as (--
            select what, nvl(ref, 'Not Identified') || ':' || median collection 
              from table_union
             order by what, ref--
        )
        select what, array_join(collect_list(trim(collection)),',')  collection
          from table_collection
        group by what
        """)
    
    dados3 = df_question_3.toPandas()
    return convert_to_Json_challenge3(dados3, question3)

# Aplicação
Função que realiza o processamento de um dataset por vez.
- Leitura do arquivo
- criação do ClickStream (sessionamento)
- dispara os algorítimos da primero, segundo e terceiro desafio
- retorna os resultados dos desafios

In [13]:
def process_dataset(filename, questions):
    question1, question2, question3 = questions
    df = read_json(filename)
    get_clickstream_dataframe(df)
    question1 = first_challenge(filename, question1)
    question2 = second_challenge(question2)
    question3 = third_challenge(question3)
    
    return question1, question2, question3


Algorítimo que realiza a leitura em loop de todos os arquivos.

In [14]:
from datetime import datetime 

questions = {}, {}, {}

for i in range(quantity_of_datasets):
    filename = 'part-0000' + str(i) +'.json.gz'
    print(filename, 'Start at', datetime.today())
    questions = process_dataset(filename, questions)

question1, question2, question3 = questions

print('Finished at', datetime.today())

part-00000.json.gz Start at 2020-08-28 22:51:14.816160
part-00001.json.gz Start at 2020-08-28 23:21:14.319939
part-00002.json.gz Start at 2020-08-28 23:53:07.234957
part-00003.json.gz Start at 2020-08-29 00:18:32.079236
part-00004.json.gz Start at 2020-08-29 00:54:56.534240
part-00005.json.gz Start at 2020-08-29 01:24:16.688486
part-00006.json.gz Start at 2020-08-29 01:45:21.289146
part-00007.json.gz Start at 2020-08-29 02:18:40.099745
part-00008.json.gz Start at 2020-08-29 04:18:02.588968
part-00009.json.gz Start at 2020-08-29 04:56:30.977899
Finished at 2020-08-29 05:25:39.735337


# Resultados

O primeiro resultado possui a quantidade de sessões unicas por arquivo de conjunto de dados em formato Json.

In [15]:
print(question1)

{'part-00000.json.gz': 10234891, 'part-00001.json.gz': 10230776, 'part-00002.json.gz': 10227212, 'part-00003.json.gz': 10233329, 'part-00004.json.gz': 10233306, 'part-00005.json.gz': 10235205, 'part-00006.json.gz': 10232658, 'part-00007.json.gz': 10236335, 'part-00008.json.gz': 10228150, 'part-00009.json.gz': 10229043}


O segundo resultado possui a quantidade de sessões únicas que ocorreram em cada Browser, Sistema Operacional e Dispositivo.

In [16]:
print(question2)

{'device_family': {'': 1001.0, '1001-G Go': 2338.0, '2014819': 803.0, '4009E': 134.0, '4009I': 73.0, '4017F': 2451.0, '4028E': 27.0, '4034E': 15214.0, '4055J': 1728.0, '5010E': 14665.0, '5016J': 973.0, '5017E': 127.0, '5026J': 2744.0, '5033E': 3997.0, '5033J': 8881.0, '5045J': 4569.0, '5046J': 937.0, '5049E': 319.0, '5051J': 2510.0, '5054W': 95.0, '5056N': 292.0, '5085J': 2252.0, '5085N': 5142.0, '5090I': 265.0, '5152D': 3805.0, '5159J': 10077.0, '5186D': 1547.0, '5199I': 900.0, '6039J': 211.0, '6055B': 2116.0, '6060S': 71.0, '62S': 554.0, '7048A': 13.0, '705-G': 237.0, '705-G Go': 4813.0, '71S': 500.0, '8050E': 17607.0, '9008J': 3824.0, '9008N': 6670.0, '91S': 43.0, 'A3_Pro': 74.0, 'ALE-L21': 720.0, 'ATU-LX3': 39.0, 'Advance 4.0 L3': 377.0, 'Advance 4.0M': 426.0, 'Advance 5.2': 201.0, 'Advance L4': 918.0, 'Aquaris X Pro': 218.0, 'Archos 60 Platinum': 193.0, 'Archos Access 50 Color 3G': 279.0, 'Archos Core 50 4G': 774.0, 'Armor_3': 5.0, 'Armor_6': 222.0, 'Asus A001D': 107105.0, 'Asus A

O terceiro resultado possui o cálculo da mediana da duração em segundos entre todas as sessões únicas. 

In [17]:
print(question3)

{'device_family': {'Generic Smartphone': 589.46581640625, 'Samsung SM-A105FN': 400.442, 'Samsung SM-G9650': 794.2095, 'Samsung SM-J415G': 403.402, 'Spider': 478.715, 'iPhone': 888.317, 'Other': 471.556140625, 'Samsung SM-J105B': 681.886, 'LM-X410.F': 3.0415, 'Samsung SM-G610M': 734.18175, 'iPhone10': 0, '5': 310.9285, 'Samsung SM-G570M': 605.681, 'Samsung SM-J610G': 1240.14625, 'XiaoMi MI 8 Pro': 117.5235, 'iPhone8': 0, '1': 128.3895, 'Moto C Plus': 402.025, 'Samsung SM-A505GT': 1014.91925, 'Samsung SM-G935F': 154.901, 'Samsung GT-I9192': 565.2825, 'Samsung SM-A705MN': 404.05, 'Samsung SM-J260M': 975.153, 'Samsung SM-A605GN': 580.2545, 'Samsung SM-J810M': 315.9825, 'XiaoMi Redmi Note 8': 712.971}, 'os_family': {'Android': 924.590958984375, 'Other': 375.03025, 'iOS': 638.087125, 'Windows 10': 740.0115625, 'Linux': 368.7685, 'Windows 8.1': 883.6665, 'Windows 7': 94.371}, 'browser_family': {'Chrome Mobile': 1147.512375, 'Facebook': 596.6268515625, 'Googlebot': 478.715, 'Mobile Safari': 88

Criação de arquivos tipo json na pasta output.

In [18]:
def save_file(filename, content):
    if not os.path.exists(outPath):
        os.makedirs(outPath)
    with open(outPath+ "/" + filename, "w") as outfile:  
        json.dump(content, outfile) 

In [19]:
save_file('question1.json', question1)
save_file('question2.json', question2)
save_file('question3.json', question3)